In [1]:
import numpy as np
import pandas as pd
import os
import json
from sklearn.cluster import KMeans
from tqdm import notebook

import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as pltoff
import plotly.figure_factory as ff


In [ ]:
path = 'shanghai/'
files = os.listdir(path)

df_list = []
df_merge = pd.DataFrame()
for file in files:
    if file == 'confirmedCases.csv':
        with open(path+file, encoding="utf-8") as f:
            confirmedCases = pd.read_csv(f)
    elif file == 'asymptomaticCases.csv':
        with open(path+file, encoding="utf-8") as f:
            asymptomaticCases = pd.read_csv(f)    
    elif '.csv' in file:
        with open(path+file, encoding="utf-8") as f:
            df = pd.read_csv(f)
            df_list.append(df)
            df_merge = pd.concat([df_merge, df])
            
def get_row_from_df(df):
    sub_df = df.drop('date')
    districts = df.keys()

## 1.分区总数

In [ ]:
def Fig_TollByDistricts(df, json_path='data/shanghai_boundary.jspn', label='无症状感染者', save=False, zmax=10000):
    token = open(".mapbox_token").read()
    with open(json_path, encoding='utf-8-sig') as f:
        shanghai_geo = json.load(f)
    color_list = [ '#2e598f',  '#9abdae', '#d3d179', '#e7c034', ]
    
    px.set_mapbox_access_token(open(".mapbox_token").read())
    fig = px.choropleth_mapbox(df,
                               featureidkey='properties.name',
                               geojson=shanghai_geo, 
                               locations='district', 
                               color='val',
                               color_continuous_scale=color_list,
                               zoom=7.5, center = {"lat": 31.23, "lon": 121.47},
                               opacity=0.7,
                               labels={'val':'上海新冠肺炎'+label+'汇总', },
                               animation_frame='date',
                               range_color=[0,zmax]
                              )
    if save:
        pltoff.plot(fig, filename=label+'.html')
    fig.show()


df = asymptomaticCases
df = pd.melt(asymptomaticCases, id_vars='date', var_name='district', value_name='val')
Fig_TollByDistricts(df, label='无症状感染者', save=True)

df = confirmedCases
df = pd.melt(confirmedCases, id_vars='date', var_name='district', value_name='val')
Fig_TollByDistricts(df, label='确诊病例', save=True, zmax=1000)

## 2.疫情点

In [ ]:
def Fig_Scatter(df, savename=None, time_line=None):
    color_list = ['#1e1e2a', '#293e5d', '#2e598f', '#397ab2', '#59a3c4', 
                  '#738a83', '#9abdae', '#e5e8b3', '#d3d179', '#e7c034', ]
    px.set_mapbox_access_token(open(".mapbox_token").read())
    hoverD = {'address':True,'date':True,'district':True}
    city_center = {'lon': np.mean(df['lons']), 'lat': np.mean(df['lats'])}
    
    
    animation_frame = None
    color='date'
    if time_line:
        animation_frame = 'date'
        color='district'
    fig = px.scatter_mapbox(df, lat="lats", lon="lons",
                            hover_name="address",hover_data=hoverD,
                            title ='上海疫情病例报告地点',
                            center=city_center,
                            zoom=8,
                            color=color,
                            color_discrete_sequence=color_list,
                            animation_frame=animation_frame,
                            opacity=0.8,
                            range_color=color_list,
                            size_max=20
                           )
    if savename:
        pltoff.plot(fig, filename=savename+'.html')
    fig.show()

Fig_Scatter(df_merge, time_line=True, savename='上海疫情病例报告地点')


## 3.疫情热力图

In [12]:
def Fig_Heatmap(df, savename=None):
    # Graw Fig
    color_list = [ '#293e5d','#2e598f',  '#9abdae', '#d3d179', '#e7c034', ]

    hoverD = {'lon': False, 'lat':False, 'centroids_dense': True, 'district':True,
              'township': True, 'neighborhood': True, 'building':True, 'streetNumber':True, 
              'date': True,}
    px.set_mapbox_access_token(open(".mapbox_token").read())
    token = open(".mapbox_token").read()
    fig = px.density_mapbox(df, lon='lon',lat='lat',
                             z='centroids_dense',
                             radius=40,
                             hover_name ='centroids_dense' ,
                             hover_data=hoverD,
                             color_continuous_scale =color_list,
                             range_color=[0, 150],
                             animation_frame='date',
                             )

    fig.update_layout(mapbox={'accesstoken': token, 'center':{'lon': 121.47, 'lat': 31.23}, 'zoom': 7.5},
                     title={'text': '上海新冠肺炎疫情病例住址热力图', 
                            'xref': 'paper', 'x':0.5})
    if savename:
        pltoff.plot(fig, filename=savename+'.html')
    fig.show()

df = pd.read_csv('shanghai/centroid/merged_centroids.csv')
Fig_Heatmap(df, savename='疫情热力图')

## 4.卫星夜光图

In [ ]:
def Fig_ntl(df, hexnum=40, savename=None):
    df['NTL'] = df['NTL']**0.6
    df['NTL'] = df['NTL'] * 24894300 / np.sum(df['NTL'])
    px.set_mapbox_access_token(open(".mapbox_token").read())
    color_list = [ '#293e5d','#2e598f',  '#9abdae', '#d3d179', '#e7c034', ]
    
    fig = ff.create_hexbin_mapbox(
        data_frame=df, lat="lats", lon="lons",
        nx_hexagon=hexnum, opacity=0.7, 
        labels={"color": "人口密度"},
        min_count=1,
        color='NTL',
        agg_func=np.sum,
        color_continuous_scale=color_list
    )
    fig.update_layout(mapbox={ 'center':{'lon': 121.47, 'lat': 31.23}, 'zoom': 7.5},
                     title={'text': '上海人口密度（来自卫星夜光图）', 
                            'xref': 'paper', 'x':0.5})
    if savename:
        pltoff.plot(fig, filename=savename+'.html')
    fig.show()

df = pd.read_csv('data/NTL.csv')
Fig_ntl(df, savename='人口密度图')